# Bert Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [ ]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



In [ ]:
# Running in colaboratory
!git clone https://github.com/vdevictor96/fabric-federated-learning
%cd fabric-federated-learning/federated-learning/
!git pull


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Initialize variables

In [ ]:
from client.model.bert_tiny import get_bert_tiny_tokenizer

MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 2
TEST_BATCH_SIZE = 8
TRAIN_SIZE = 0.8
EVAL_SIZE = 0.2
NUM_EPOCHS = 10
LEARNING_RATE = 1e-05
SEED = 200
tokenizer = get_bert_tiny_tokenizer()
twitter_dep_train = 'client/data/datasets/twitter_dep/twitter_dep_train.csv'
twitter_dep_test = 'client/data/datasets/twitter_dep/twitter_dep_test.csv'



### Loading Reddit Depression Dataloaders

In [ ]:
from client.data.twitter_dep import get_twitter_dep_dataloaders


train_loader, eval_loader = get_twitter_dep_dataloaders(twitter_dep_train, tokenizer, 
                                                                    train_size=TRAIN_SIZE,
                                                                    eval_size=EVAL_SIZE,
                                                                    train_batch_size=TRAIN_BATCH_SIZE, 
                                                                    eval_batch_size=EVAL_BATCH_SIZE, 
                                                                    max_len=MAX_LEN, 
                                                                    seed=SEED)


### Creating Bert Tiny Model

In [ ]:
from client.model.bert_tiny import get_bert_tiny_model

bert_tiny = get_bert_tiny_model(device=device)
print(bert_tiny)


In [ ]:
# TODO necessary?
!pip install evaluate
!pip install accelerate
!pip install transformers[torch]
!pip show accelerate


### Loading Transformers Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from client.data.twitter_dep import get_twitter_dep_datasets, get_twitter_dep_test_dataset

train_dataset, eval_dataset = get_twitter_dep_datasets(twitter_dep_train, tokenizer, TRAIN_SIZE, EVAL_SIZE, MAX_LEN, SEED)
test_dataset = get_twitter_dep_test_dataset(twitter_dep_test, tokenizer, MAX_LEN, SEED)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="client/models/bert-tiny/", evaluation_strategy="epoch")

training_args = TrainingArguments(
    output_dir="client/models/bert-tiny/",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=bert_tiny,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

### Train model with Trainer

In [ ]:
trainer.train()

### Train model

In [ ]:
from torch.optim import Adam
from transformers import get_scheduler


# Loss function
loss_function = torch.nn.CrossEntropyLoss()
# Optimizer and learning rate scheduler
optimizer = Adam(bert_tiny.parameters(), lr=LEARNING_RATE)


num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

### Test the trained model

In [ ]:
from client.test import test_text_class
# Test the model
test_text_class(bert_tiny, test_loader, device=device)

# Hyperparameter search

In [ ]:
! pip install optuna
# ! pip install ray[tune]


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from client.data.twitter_dep import get_twitter_dep_datasets, get_twitter_dep_test_dataset
from client.model.bert_tiny import get_bert_tiny_tokenizer

train_dataset, eval_dataset = get_twitter_dep_datasets(twitter_dep_train, tokenizer, TRAIN_SIZE, EVAL_SIZE, MAX_LEN, SEED)
test_dataset = get_twitter_dep_test_dataset(twitter_dep_test, tokenizer, MAX_LEN, SEED)
tokenizer = get_bert_tiny_tokenizer()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="client/models/bert-tiny/", evaluation_strategy="epoch")

training_args = TrainingArguments(
    output_dir="client/models/bert-tiny/",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

def model_init():
  return AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")    

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 15),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

# best_run = trainer.hyperparameter_search(hp_space=my_hp_space, n_trials=20, direction="maximize")
best_run = trainer.hyperparameter_search(n_trials=20, direction="maximize")